In [42]:
import re
import numpy as np
import pandas as pd
from db import Agoda_Activities_Reviews
from db_traveloka import Activities_Reviews

In [29]:
# Lấy 40,000 documents đầu tiên
traveloka_documents = list(Activities_Reviews.find().limit(16978))

# In ra số lượng documents lấy được
print(f"Đã lấy {len(traveloka_documents)} documents.")

# Giả sử bạn đã có biến documents
content_list = [doc.get("Content", "") for doc in traveloka_documents]

# In thử 5 dòng đầu
for i, content in enumerate(content_list[:5], 1):
    print(f"{i}. {content}")

traveloka_cleaned_reviews = [text.replace("Đọc thêm", "").strip() for text in content_list]

# Kiểm tra kết quả
for i, review in enumerate(traveloka_cleaned_reviews[:5], 1):
    print(f"{i}. {review}\n")

Đã lấy 16978 documents.
1. Giá đặt online rẻ, nhiều trò chơi cảm giác mạnh, sẽ còn đi lại nhiều lần.
2. Đội ngũ nhân viên luôn sẵn sàng giúp đỡ.
3. tệ ..đi 1 lần k có lần sau ..người ta muốn đi tắm nó chở đi lòng vòng tham quan mấy chổ tao lao ...nhân viên thái độ ...quá tệ
4. Trò chơi ít, không như tóp tóp review, tương đối chán, chưa thật sự đáng 441.000đ.

Đi đầm sen đi :)
5. Khu nước:
Nhân viên nhiệt tình, chu đáo, dễ thương ( nước+ khô )
Còn về trò nước á thì không gian được cái to mà  chỉ có vài trò à k đa dạng 
Phao 4 người thì đc mang lên sẵn , phao 2 thì có cái bệ để mình mang lên đỡ nặng 
Trò sóng biển thì lâu lắm mới có 1 lần kiểu đông đông thì mới có sóng mà mỗi lần sóng đánh chỉ có 1 lần riêng mình thâý hơi nhàm
 Dù có mái che thật nhưng hầu như nhiều chỗ vẫn bị nắng nhìu á hôm mình đi xong rộm chân lột da đc luôn á, tr ơi đi mà đất nó nóng nó rát điên người luôn á :((( 
==> Túm lại khu nước lớn nhưng ko đa dạng trò chơi 
Khu Khô :
 Cũng thú dị có Vài Trò như nhà tuyết, 18

In [43]:
from langdetect import detect
from langdetect import detect, LangDetectException

def detect_language_safe(text):
    if not text or text.strip() == "":
        return "unknown"
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

In [44]:
documents = list(Agoda_Activities_Reviews.find().limit(51000))
df_agoda = pd.DataFrame (documents).drop (['Date', 'Time', '_id', 'Activity_Id', 'SourcePlatform'], axis=1)
print (df_agoda.duplicated (subset='Content').sum ())
df_agoda = df_agoda.drop_duplicates ()

bad_keywords = [
    "\nHiện nhiều hơn"
]

# Hàm kiểm tra xem nội dung có sạch không
def is_clean(text):
    text = text.strip()
    starts_with_thank_you = re.match(r'^thank you', text, flags=re.IGNORECASE) is not None
    contains_bad_keyword = any(keyword in text for keyword in bad_keywords)
    return not (starts_with_thank_you or contains_bad_keyword)

df_agoda = df_agoda[df_agoda['Content'].apply(is_clean)].reset_index(drop=True)
df_agoda['Content'] = df_agoda['Content'].str.replace("\nHiện ít hơn", "", regex=False).str.strip()

df_agoda['language'] = df_agoda['Content'].apply(detect_language_safe)

df_agoda

3427


,Title,Content,Per_Rating,Time_Review,Source,language
0,Dịch vụ tuyệt vời,Mọi thứ đều rất bài bản và tuyệt. Xứng đáng vớ...,5/5 trên KKday,Nhận xét vào Tháng 5 2023 trên KKday,Do KKday thu thập,vi
1,tham quan,"Thức ăn ngon, dịch vụ tại thuyền tốt. Thuyền h...",4/5 trên KKday,Nhận xét vào Tháng 7 2019 trên KKday,Do KKday thu thập,vi
2,最高のディナークルーズ,ビュッフェだけでなく、歌のショーやベリーダンスなど楽しませるイベントも盛りだくさんで、すごく...,5/5 trên KKday,Nhận xét vào Tháng 6 2025 trên KKday,Do KKday thu thập,ja
3,值得參加,餐點好吃 有表演 夜景很美 來越南很值得參加郵輪晚餐👍,5/5 trên KKday,Nhận xét vào Tháng 6 2025 trên KKday,Do KKday thu thập,ko
4,東西好吃又好玩,原本看評論有點害怕 採雷 食材好吃 氣氛好 剛好有印度人來這慶生 表演者讓全場氣氛變的很hi...,5/5 trên KKday,Nhận xét vào Tháng 5 2025 trên KKday,Do KKday thu thập,ko
...,...,...,...,...,...,...
34788,Full day experience,Die Tour war super! Bin war unser Guide. Ein s...,5/5 trên Tripadvisor,Nhận xét vào Tháng 2 2024 trên Tripadvisor,Do Tripadvisor thu thập,de
34789,Cu Chi Tunnel & Mekong Delta Day Tour Experience,"The service provided was excellent, even with ...",5/5 trên Tripadvisor,Nhận xét vào Tháng 2 2024 trên Tripadvisor,Do Tripadvisor thu thập,en
34790,Fantastic 😊,Mai's sense of humor and friendliness are trul...,5/5 trên Tripadvisor,Nhận xét vào Tháng 2 2024 trên Tripadvisor,Do Tripadvisor thu thập,en
34791,A wonderful experience lots of fun,I had an excellent talk with a wonderful guide...,5/5 trên Tripadvisor,Nhận xét vào Tháng 1 2024 trên Tripadvisor,Do Tripadvisor thu thập,en


In [46]:
df_vi_agoda = df_agoda.loc [df_agoda ['language'] == 'vi']
df_vi_agoda = df_vi_agoda.drop_duplicates (subset='Content')
df_vi_agoda

,Title,Content,Per_Rating,Time_Review,Source,language
0,Dịch vụ tuyệt vời,Mọi thứ đều rất bài bản và tuyệt. Xứng đáng vớ...,5/5 trên KKday,Nhận xét vào Tháng 5 2023 trên KKday,Do KKday thu thập,vi
1,tham quan,"Thức ăn ngon, dịch vụ tại thuyền tốt. Thuyền h...",4/5 trên KKday,Nhận xét vào Tháng 7 2019 trên KKday,Do KKday thu thập,vi
472,Great tour!,Tour được tổ chức chu đáo. Anh hướng dẫn viên ...,5/5 trên Tripadvisor,Nhận xét vào Tháng 6 2024 trên Tripadvisor,Do Tripadvisor thu thập,vi
473,Chuyến đi gia đình cùng hướng dẫn viên Bảo,Có một chuyến đi rất vui và nhiều niềm vui cùn...,5/5 trên Tripadvisor,Nhận xét vào Tháng 2 2023 trên Tripadvisor,Do Tripadvisor thu thập,vi
474,Tour tuyệt vời,Anh Lộc (lockie) hướng dẫn rất vui tính và tận...,5/5 trên Tripadvisor,Nhận xét vào Tháng 8 2022 trên Tripadvisor,Do Tripadvisor thu thập,vi
...,...,...,...,...,...,...
31435,Mekong Delta Day Tour,The review was excellent. We have a lot of fun...,5/5 trên Tripadvisor,Nhận xét vào Tháng 11 2024 trên Tripadvisor,Do Tripadvisor thu thập,vi
32918,Deaf tour in Vietnam was fantastic... Coda rev...,After visa problems delayed me 3 hours the tou...,5/5 trên Tripadvisor,Nhận xét vào Tháng 7 2018 trên Tripadvisor,Do Tripadvisor thu thập,vi
33069,Very good,"Hướng dẫn viên chuyên nghiệp, nhiệt tình , qua...",5/5 trên Tripadvisor,Nhận xét vào Tháng 8 2023 trên Tripadvisor,Do Tripadvisor thu thập,vi
33353,Mekong delta,"HDV Tánh đã phục vụ rất ân cần, quan tâm tận t...",5/5 trên Tripadvisor,Nhận xét vào Tháng 4 2023 trên Tripadvisor,Do Tripadvisor thu thập,vi


In [47]:
traveloka_documents = list(Activities_Reviews.find().limit(16978))
df = pd.DataFrame (traveloka_documents)
df.drop (['Date', 'Time', '_id', 'Activity_Id', 'SourcePlatform'], axis=1, inplace=True)
print (df.duplicated ().sum ())
df['Content'] = df['Content'].str.replace("Đọc thêm", "", regex=False).str.strip()
df['language'] = df['Content'].apply(detect_language_safe)
df = df.drop_duplicates (subset='Content')
df

5767


,Author,Content,Per_Rating,Time_Review,Likes,language
0,28 Nguyễn N.,"Giá đặt online rẻ, nhiều trò chơi cảm giác mạn...","8,0",Đánh giá cách đây 5 ngày,0,vi
1,T***g,Đội ngũ nhân viên luôn sẵn sàng giúp đỡ.,"8,0",Đánh giá cách đây 1 tuần,0,vi
2,Tran t. H. l.,tệ ..đi 1 lần k có lần sau ..người ta muốn đi ...,"4,0",Đánh giá cách đây 2 tuần,0,vi
3,vietanh l.,"Trò chơi ít, không như tóp tóp review, tương đ...","4,0",Đánh giá cách đây 2 tuần,1,vi
4,N***t,"Khu nước:\nNhân viên nhiệt tình, chu đáo, dễ t...","6,0",Đánh giá cách đây 2 tuần,0,vi
...,...,...,...,...,...,...
16968,Leo R.,Chuyến đi mình mong chờ nhất. Tuy nhiên thời g...,"10,0",Đánh giá cách đây 294 tuần,6,vi
16969,Ba Quan M.,"Ra đảo đúng ngày biển động, sóng to, gió lớn. ...","5,0",Đánh giá cách đây 297 tuần,4,vi
16970,Thanh T. N.,Trời mưa bão nhưng bên hãng nói vẫn đi được. S...,"4,0",Đánh giá cách đây 300 tuần,0,vi
16971,Freshi M.,"Tài xế rất nhiệt tình, trước khi đón H-1 tài x...","10,0",Đánh giá cách đây 34 tuần,0,vi


In [48]:
df_vi_traveloka = df.loc [df ['language'] == 'vi']
# df_vi_traveloka = df_vi_traveloka.drop_duplicates (subset='Content')
df_vi_traveloka

,Author,Content,Per_Rating,Time_Review,Likes,language
0,28 Nguyễn N.,"Giá đặt online rẻ, nhiều trò chơi cảm giác mạn...","8,0",Đánh giá cách đây 5 ngày,0,vi
1,T***g,Đội ngũ nhân viên luôn sẵn sàng giúp đỡ.,"8,0",Đánh giá cách đây 1 tuần,0,vi
2,Tran t. H. l.,tệ ..đi 1 lần k có lần sau ..người ta muốn đi ...,"4,0",Đánh giá cách đây 2 tuần,0,vi
3,vietanh l.,"Trò chơi ít, không như tóp tóp review, tương đ...","4,0",Đánh giá cách đây 2 tuần,1,vi
4,N***t,"Khu nước:\nNhân viên nhiệt tình, chu đáo, dễ t...","6,0",Đánh giá cách đây 2 tuần,0,vi
...,...,...,...,...,...,...
16968,Leo R.,Chuyến đi mình mong chờ nhất. Tuy nhiên thời g...,"10,0",Đánh giá cách đây 294 tuần,6,vi
16969,Ba Quan M.,"Ra đảo đúng ngày biển động, sóng to, gió lớn. ...","5,0",Đánh giá cách đây 297 tuần,4,vi
16970,Thanh T. N.,Trời mưa bão nhưng bên hãng nói vẫn đi được. S...,"4,0",Đánh giá cách đây 300 tuần,0,vi
16971,Freshi M.,"Tài xế rất nhiệt tình, trước khi đón H-1 tài x...","10,0",Đánh giá cách đây 34 tuần,0,vi


In [50]:
df_vi = pd.concat([df_vi_agoda, df_vi_traveloka], ignore_index=True, sort=True)
df_vi

,Author,Content,Likes,Per_Rating,Source,Time_Review,Title,language
0,NaN,Mọi thứ đều rất bài bản và tuyệt. Xứng đáng vớ...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 5 2023 trên KKday,Dịch vụ tuyệt vời,vi
1,NaN,"Thức ăn ngon, dịch vụ tại thuyền tốt. Thuyền h...",NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 7 2019 trên KKday,tham quan,vi
2,NaN,Tour được tổ chức chu đáo. Anh hướng dẫn viên ...,NaN,5/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 6 2024 trên Tripadvisor,Great tour!,vi
3,NaN,Có một chuyến đi rất vui và nhiều niềm vui cùn...,NaN,5/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 2 2023 trên Tripadvisor,Chuyến đi gia đình cùng hướng dẫn viên Bảo,vi
4,NaN,Anh Lộc (lockie) hướng dẫn rất vui tính và tận...,NaN,5/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 8 2022 trên Tripadvisor,Tour tuyệt vời,vi
...,...,...,...,...,...,...,...,...
10071,Leo R.,Chuyến đi mình mong chờ nhất. Tuy nhiên thời g...,6.0,"10,0",NaN,Đánh giá cách đây 294 tuần,NaN,vi
10072,Ba Quan M.,"Ra đảo đúng ngày biển động, sóng to, gió lớn. ...",4.0,"5,0",NaN,Đánh giá cách đây 297 tuần,NaN,vi
10073,Thanh T. N.,Trời mưa bão nhưng bên hãng nói vẫn đi được. S...,0.0,"4,0",NaN,Đánh giá cách đây 300 tuần,NaN,vi
10074,Freshi M.,"Tài xế rất nhiệt tình, trước khi đón H-1 tài x...",0.0,"10,0",NaN,Đánh giá cách đây 34 tuần,NaN,vi


In [51]:
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoConfig,
    RobertaForSequenceClassification,
)

# ========== Load tất cả model & tokenizer ==========
# 1. ViSoBERT
visobert_path = '5CD-AI/Vietnamese-Sentiment-visobert'
visobert_tokenizer = AutoTokenizer.from_pretrained(visobert_path)
visobert_config = AutoConfig.from_pretrained(visobert_path)
visobert_model = AutoModelForSequenceClassification.from_pretrained(visobert_path)
visobert_model.eval()

# 2. wonrax/phobert (chú ý: phải word-segmented sẵn)
wonrax_path = 'wonrax/phobert-base-vietnamese-sentiment'
wonrax_tokenizer = AutoTokenizer.from_pretrained(wonrax_path, use_fast=False)
wonrax_model = RobertaForSequenceClassification.from_pretrained(wonrax_path)
wonrax_model.eval()
wonrax_labels = ['NEG', 'POS', 'NEU']

# 3. mr4/phobert
mr4_path = 'mr4/phobert-base-vi-sentiment-analysis'
mr4_tokenizer = AutoTokenizer.from_pretrained(mr4_path)
mr4_model = AutoModelForSequenceClassification.from_pretrained(mr4_path)
mr4_model.eval()

# 3. Multilingual sentiment (5 classes)
multi_tokenizer = AutoTokenizer.from_pretrained('tabularisai/multilingual-sentiment-analysis')
multi_model = AutoModelForSequenceClassification.from_pretrained('tabularisai/multilingual-sentiment-analysis')
multi_model.eval()
# Bản đồ 5 nhãn về 3 nhãn
multi_map = {
    "Very Negative": "NEG",
    "Negative": "NEG",
    "Neutral":  "NEU",
    "Positive": "POS",
    "Very Positive": "POS"
}


# ========== Hàm dự đoán từ 3 mô hình ==========
def predict_with_all_models(sentence):
    results = []

    # 1. ViSoBERT
    inputs = visobert_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = visobert_model(**inputs)
        scores = outputs.logits.softmax(dim=-1).cpu().numpy()[0]
        label = visobert_config.id2label[np.argmax(scores)]
        results.append(label)

    # 2. wonrax/phobert (input đã word-segmented, nhưng tạm thời vẫn dùng nguyên câu)
    inputs = wonrax_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = wonrax_model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
        label = wonrax_labels[probs.argmax().item()]
        results.append(label)

    # 3. mr4/phobert
    # inputs = mr4_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    # with torch.no_grad():
    #     outputs = mr4_model(**inputs)
    #     probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
    #     label = mr4_model.config.id2label[probs.argmax().item()]
    #     if (label == 'Tích cực'):
    #         label = 'POS'
    #     elif (label == 'Tiêu cực'):
    #         label = 'NEG'
    #     else:
    #         label = 'NEU'
    #     results.append(label)
    
    # Multilingual model
    inputs = multi_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = multi_model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
        raw_label = multi_model.config.id2label[probs.argmax().item()]
        multi_map = {
            "Very Negative": "NEG",
            "Negative": "NEG",
            "Neutral":  "NEU",
            "Positive": "POS",
            "Very Positive": "POS"
        }
        label = multi_map.get(raw_label, "NEU")
        results.append(label)


    return results

# ===== Hàm xử lý cả danh sách review và trả về DataFrame =====
def predict_reviews_df(df_input, text_column='Content'):
    df = df_input.copy()  # tránh sửa trực tiếp
    df['labels'] = df[text_column].apply(predict_with_all_models)
    return df

df_vi = predict_reviews_df(df_vi)
df_vi

,Author,Content,Likes,Per_Rating,Source,Time_Review,Title,language,labels
0,NaN,Mọi thứ đều rất bài bản và tuyệt. Xứng đáng vớ...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 5 2023 trên KKday,Dịch vụ tuyệt vời,vi,"[POS, POS, POS]"
1,NaN,"Thức ăn ngon, dịch vụ tại thuyền tốt. Thuyền h...",NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 7 2019 trên KKday,tham quan,vi,"[POS, POS, POS]"
2,NaN,Tour được tổ chức chu đáo. Anh hướng dẫn viên ...,NaN,5/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 6 2024 trên Tripadvisor,Great tour!,vi,"[POS, POS, POS]"
3,NaN,Có một chuyến đi rất vui và nhiều niềm vui cùn...,NaN,5/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 2 2023 trên Tripadvisor,Chuyến đi gia đình cùng hướng dẫn viên Bảo,vi,"[POS, POS, POS]"
4,NaN,Anh Lộc (lockie) hướng dẫn rất vui tính và tận...,NaN,5/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 8 2022 trên Tripadvisor,Tour tuyệt vời,vi,"[POS, POS, POS]"
...,...,...,...,...,...,...,...,...,...
10071,Leo R.,Chuyến đi mình mong chờ nhất. Tuy nhiên thời g...,6.0,"10,0",NaN,Đánh giá cách đây 294 tuần,NaN,vi,"[NEU, POS, NEU]"
10072,Ba Quan M.,"Ra đảo đúng ngày biển động, sóng to, gió lớn. ...",4.0,"5,0",NaN,Đánh giá cách đây 297 tuần,NaN,vi,"[NEG, POS, NEU]"
10073,Thanh T. N.,Trời mưa bão nhưng bên hãng nói vẫn đi được. S...,0.0,"4,0",NaN,Đánh giá cách đây 300 tuần,NaN,vi,"[NEG, NEG, NEU]"
10074,Freshi M.,"Tài xế rất nhiệt tình, trước khi đón H-1 tài x...",0.0,"10,0",NaN,Đánh giá cách đây 34 tuần,NaN,vi,"[POS, POS, POS]"


In [52]:
from collections import Counter

# Hàm phân loại loại nhãn
def classify_label_type(label_list):
    counts = Counter(label_list)
    unique = len(counts)
    if unique == 1:
        return "same"
    elif unique == 2:
        return "majority"
    else:
        return "all_diff"

def add_label_type_column(df_vi):
    df = df_vi.copy()
    df['label_type'] = df['labels'].apply(classify_label_type)
    return df

df_labeled = add_label_type_column(df_vi)  # tạo thêm cột 'label_type'
# # Tách thành 3 DataFrame
# df_same = df[df['label_type'] == 'same'].copy()
# df_majority = df[df['label_type'] == 'majority'].copy()
# df_all_diff = df[df['label_type'] == 'all_diff'].copy()

In [53]:
df_labeled.to_csv ('vi_reviews.csv')

In [58]:
df_labeled

,Author,Content,Likes,Per_Rating,Source,Time_Review,Title,language,labels,label_type
0,NaN,Mọi thứ đều rất bài bản và tuyệt. Xứng đáng vớ...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 5 2023 trên KKday,Dịch vụ tuyệt vời,vi,"[POS, POS, POS]",same
1,NaN,"Thức ăn ngon, dịch vụ tại thuyền tốt. Thuyền h...",NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 7 2019 trên KKday,tham quan,vi,"[POS, POS, POS]",same
2,NaN,Tour được tổ chức chu đáo. Anh hướng dẫn viên ...,NaN,5/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 6 2024 trên Tripadvisor,Great tour!,vi,"[POS, POS, POS]",same
3,NaN,Có một chuyến đi rất vui và nhiều niềm vui cùn...,NaN,5/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 2 2023 trên Tripadvisor,Chuyến đi gia đình cùng hướng dẫn viên Bảo,vi,"[POS, POS, POS]",same
4,NaN,Anh Lộc (lockie) hướng dẫn rất vui tính và tận...,NaN,5/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 8 2022 trên Tripadvisor,Tour tuyệt vời,vi,"[POS, POS, POS]",same
...,...,...,...,...,...,...,...,...,...,...
10071,Leo R.,Chuyến đi mình mong chờ nhất. Tuy nhiên thời g...,6.0,"10,0",NaN,Đánh giá cách đây 294 tuần,NaN,vi,"[NEU, POS, NEU]",majority
10072,Ba Quan M.,"Ra đảo đúng ngày biển động, sóng to, gió lớn. ...",4.0,"5,0",NaN,Đánh giá cách đây 297 tuần,NaN,vi,"[NEG, POS, NEU]",all_diff
10073,Thanh T. N.,Trời mưa bão nhưng bên hãng nói vẫn đi được. S...,0.0,"4,0",NaN,Đánh giá cách đây 300 tuần,NaN,vi,"[NEG, NEG, NEU]",majority
10074,Freshi M.,"Tài xế rất nhiệt tình, trước khi đón H-1 tài x...",0.0,"10,0",NaN,Đánh giá cách đây 34 tuần,NaN,vi,"[POS, POS, POS]",same


In [61]:
a = df_labeled[df_labeled['labels'].apply(lambda x: x == ['NEU', 'NEU', 'NEU'])]
a

,Author,Content,Likes,Per_Rating,Source,Time_Review,Title,language,labels,label_type
15,NaN,Bình thường thôi. Trẻ em thì thích,NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 7 2024 trên KKday,cũng ok,vi,"[NEU, NEU, NEU]",same
62,NaN,Đặt vé 18h nhưng 18h20 xe mới khởi hành. Trên ...,NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 2 2021 trên KKday,Tạm ổn,vi,"[NEU, NEU, NEU]",same
208,Le Mai,Mua vé oline nhưng đến cổng phải qui đổi vé. ...,0.0,"8,0",NaN,Đánh giá cách đây 57 tuần,NaN,vi,"[NEU, NEU, NEU]",same
322,Le van l.,Bình thường.,0.0,"6,0",NaN,Đánh giá cách đây 293 tuần,NaN,vi,"[NEU, NEU, NEU]",same
399,Nguyen M. T.,"Tạm ổn, nhưng ko đa dạng\nDành cho trẻ em thì ok",1.0,"8,0",NaN,Đánh giá cách đây 29 tuần,NaN,vi,"[NEU, NEU, NEU]",same
...,...,...,...,...,...,...,...,...,...,...
9749,Tran Q. H.,nó là đẹp nhưng trình bày rất nhỏ của động vật.,3.0,"10,0",NaN,Đánh giá cách đây 167 tuần,NaN,vi,"[NEU, NEU, NEU]",same
9923,Phuoc P.,Tour tốt nhưng hơi chán vì không nhiều điều th...,1.0,"8,0",NaN,Đánh giá cách đây 133 tuần,NaN,vi,"[NEU, NEU, NEU]",same
9992,Nguyen D. K.,"Câu mực nhưng không câu được gì, bữa tối không...",2.0,"6,0",NaN,Đánh giá cách đây 77 tuần,NaN,vi,"[NEU, NEU, NEU]",same
9996,Hoang A. N.,Tạm ổn. Cần cải thiện bữa tối hơn.,4.0,"8,0",NaN,Đánh giá cách đây 232 tuần,NaN,vi,"[NEU, NEU, NEU]",same


In [55]:
a = df_labeled.loc [df_labeled ['label_type'] == 'all_diff']
a

,Author,Content,Likes,Per_Rating,Source,Time_Review,Title,language,labels,label_type
110,NaN,Đúng là phải đi 1 lần cho đáng 1 lần. Ai từng ...,NaN,4/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 2 2020 trên Tripadvisor,Tự hào về địa đạo Củ Chi,vi,"[NEU, POS, NEG]",all_diff
130,vietanh l.,"Trò chơi ít, không như tóp tóp review, tương đ...",1.0,"4,0",NaN,Đánh giá cách đây 2 tuần,NaN,vi,"[NEG, POS, NEU]",all_diff
134,P***N,"Dịch vụ ok nhất là ăn uống, không quá mắc, hợp...",0.0,"6,0",NaN,Đánh giá cách đây 4 tuần,NaN,vi,"[NEG, POS, NEU]",all_diff
186,Phuong N. N.,mọi thứ vẫn tuyệt nhưng mà hôm nay mình đi mới...,0.0,"8,0",NaN,Đánh giá cách đây 47 tuần,NaN,vi,"[NEU, POS, NEG]",all_diff
201,Alice D.,Mình đi team 7 người và cảm thấy số lượng trò ...,0.0,"8,0",NaN,Đánh giá cách đây 54 tuần,NaN,vi,"[NEU, POS, NEG]",all_diff
...,...,...,...,...,...,...,...,...,...,...
10027,huynh t. n. u.,"Đi grand thực tế không cần mua vé vào, những v...",5.0,"6,0",NaN,Đánh giá cách đây 118 tuần,NaN,vi,"[NEG, POS, NEU]",all_diff
10045,Tran M. V.,"Khu này không rộng lắm, nên đi vào chiều tối đ...",2.0,"8,0",NaN,Đánh giá cách đây 167 tuần,NaN,vi,"[NEU, POS, NEG]",all_diff
10047,NGUYEN H. N.,Bạn có thể mua vé tại các cổng với giá tương đ...,1.0,"8,0",NaN,Đánh giá cách đây 180 tuần,NaN,vi,"[NEG, POS, NEU]",all_diff
10070,Ha Ta,Tour 2 đảo mà thấy lên được có 1 đảo. Có chươn...,2.0,"6,0",NaN,Đánh giá cách đây 293 tuần,NaN,vi,"[NEG, POS, NEU]",all_diff


In [11]:
df_en_agoda = df_agoda.loc [df_agoda ['language'] == 'en']
print ('English Agoda:', len (df_en_agoda))
df_en_traveloka = df.loc [df ['language'] == 'en']
print ('English Traveloka:', len (df_en_traveloka))
df_en = pd.concat([df_en_agoda, df_en_traveloka], ignore_index=True, sort=True)
print ('English both:', len (df_en))

English Agoda: 28422
English Traveloka: 538
English both: 28960


In [17]:
df_en

,Author,Content,Likes,Per_Rating,Source,Time_Review,Title,language
0,NaN,I appreciate that catering for everyone’s tast...,NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 4 2025 trên KKday,City lights river cruise,en
1,NaN,"The staff was good, the buffet is quite extens...",NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 2 2025 trên KKday,Old boat but it was fun,en
2,NaN,The buffet served onboard got lots of variety ...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 2 2025 trên KKday,Excellent,en
3,NaN,Buffet dinner was good and we enjoyed the beau...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 8 2024 trên KKday,Great tour,en
4,NaN,Seafood very fresh and nice,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 7 2024 trên KKday,delicious,en
...,...,...,...,...,...,...,...,...
28955,Ng Kok K.,The guides were extremely helpful and friendly...,0.0,"10,0",NaN,Đánh giá cách đây 216 tuần,NaN,en
28956,Quang H.,Nice services. Proud of Tinh Hoa Vietnam show,0.0,"10,0",NaN,Đánh giá cách đây 169 tuần,NaN,en
28957,Van T. H. T.,Great experience. I loved it so much.,0.0,"10,0",NaN,Đánh giá cách đây 238 tuần,NaN,en
28958,Ngan,We were able to be picked up at our own resort...,0.0,"9,0",NaN,Đánh giá cách đây 286 tuần,NaN,en


In [21]:
import torch
import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
)
from collections import Counter

# ========== Load mô hình tiếng Anh ==========
# 1. Twitter-RoBERTa
roberta_en_path = 'cardiffnlp/twitter-roberta-base-sentiment'
roberta_en_tokenizer = AutoTokenizer.from_pretrained(roberta_en_path)
roberta_en_model = AutoModelForSequenceClassification.from_pretrained(roberta_en_path)
roberta_en_model.eval()
roberta_en_labels = ['NEG', 'NEU', 'POS']

# 2. BERT-based model cho review dài
nlptown_path = "nlptown/bert-base-multilingual-uncased-sentiment"
nlptown_tokenizer = AutoTokenizer.from_pretrained(nlptown_path)
nlptown_model = AutoModelForSequenceClassification.from_pretrained(nlptown_path)
nlptown_model.eval()

# Bản đồ: 1–2 sao → NEG, 3 sao → NEU, 4–5 sao → POS
nlptown_map = {
    0: "NEG",
    1: "NEG",
    2: "NEU",
    3: "POS",
    4: "POS"
}


# 3. Multilingual (map 5 nhãn về 3)
# multi_path = 'tabularisai/multilingual-sentiment-analysis'
# multi_tokenizer = AutoTokenizer.from_pretrained(multi_path)
# multi_model = AutoModelForSequenceClassification.from_pretrained(multi_path)
# multi_model.eval()
# multi_map = {
#     "Very Negative": "NEG",
#     "Negative": "NEG",
#     "Neutral":  "NEU",
#     "Positive": "POS",
#     "Very Positive": "POS"
# }

# ========== Hàm dự đoán từ 3 mô hình ==========
def predict_with_all_models_en(sentence):
    try:
        if not isinstance(sentence, str) or sentence.strip() == "":
            return ['NEU']

        results = []

        # 1. Twitter-RoBERTa
        try:
            inputs = roberta_en_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
            inputs = {k: v.to(roberta_en_model.device) for k, v in inputs.items()}
            with torch.no_grad():
                outputs = roberta_en_model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
                label = roberta_en_labels[probs.argmax().item()]
                results.append(label)
        except Exception as e:
            print("[Roberta Error]", sentence[:100], e)
            results.append("NEU")

        # 2. nlptown sentiment (5 sao)
        try:
            inputs = nlptown_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
            inputs = {k: v.to(nlptown_model.device) for k, v in inputs.items()}
            with torch.no_grad():
                outputs = nlptown_model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
                label_idx = probs.argmax().item()
                label = nlptown_map.get(label_idx, "NEU")
                results.append(label)
        except Exception as e:
            print("[NLPtown Error]", sentence[:100], e)
            results.append("NEU")

        # (Optional) Nếu muốn thêm lại multilingual thì mở comment đoạn dưới
        # ...

        return results

    except Exception as e:
        print("[Unknown Error]:", sentence[:100], e)
        return ['NEU']

# ===== Hàm xử lý DataFrame reviews tiếng Anh =====
def predict_reviews_df_en(df_input, text_column='Content'):
    df = df_input.copy()
    df['labels'] = df[text_column].apply(predict_with_all_models_en)
    return df

df_en1 = predict_reviews_df_en(df_en)
df_en1

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

,Author,Content,Likes,Per_Rating,Source,Time_Review,Title,language,labels
0,NaN,I appreciate that catering for everyone’s tast...,NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 4 2025 trên KKday,City lights river cruise,en,"[POS, POS]"
1,NaN,"The staff was good, the buffet is quite extens...",NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 2 2025 trên KKday,Old boat but it was fun,en,"[POS, POS]"
2,NaN,The buffet served onboard got lots of variety ...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 2 2025 trên KKday,Excellent,en,"[POS, POS]"
3,NaN,Buffet dinner was good and we enjoyed the beau...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 8 2024 trên KKday,Great tour,en,"[POS, POS]"
4,NaN,Seafood very fresh and nice,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 7 2024 trên KKday,delicious,en,"[POS, POS]"
...,...,...,...,...,...,...,...,...,...
28955,Ng Kok K.,The guides were extremely helpful and friendly...,0.0,"10,0",NaN,Đánh giá cách đây 216 tuần,NaN,en,"[POS, POS]"
28956,Quang H.,Nice services. Proud of Tinh Hoa Vietnam show,0.0,"10,0",NaN,Đánh giá cách đây 169 tuần,NaN,en,"[POS, POS]"
28957,Van T. H. T.,Great experience. I loved it so much.,0.0,"10,0",NaN,Đánh giá cách đây 238 tuần,NaN,en,"[POS, POS]"
28958,Ngan,We were able to be picked up at our own resort...,0.0,"9,0",NaN,Đánh giá cách đây 286 tuần,NaN,en,"[POS, POS]"


In [25]:
def classify_label_type(label_list):
    counts = Counter(label_list)
    unique = len(counts)
    if unique == 1:
        return "same"
    elif unique == 2:
        return "majority"
    else:
        return "all_diff"
    
df_labeled_en = add_label_type_column(df_en1) 
df_labeled_en

,Author,Content,Likes,Per_Rating,Source,Time_Review,Title,language,labels,label_type
0,NaN,I appreciate that catering for everyone’s tast...,NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 4 2025 trên KKday,City lights river cruise,en,"[POS, POS]",same
1,NaN,"The staff was good, the buffet is quite extens...",NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 2 2025 trên KKday,Old boat but it was fun,en,"[POS, POS]",same
2,NaN,The buffet served onboard got lots of variety ...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 2 2025 trên KKday,Excellent,en,"[POS, POS]",same
3,NaN,Buffet dinner was good and we enjoyed the beau...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 8 2024 trên KKday,Great tour,en,"[POS, POS]",same
4,NaN,Seafood very fresh and nice,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 7 2024 trên KKday,delicious,en,"[POS, POS]",same
...,...,...,...,...,...,...,...,...,...,...
28955,Ng Kok K.,The guides were extremely helpful and friendly...,0.0,"10,0",NaN,Đánh giá cách đây 216 tuần,NaN,en,"[POS, POS]",same
28956,Quang H.,Nice services. Proud of Tinh Hoa Vietnam show,0.0,"10,0",NaN,Đánh giá cách đây 169 tuần,NaN,en,"[POS, POS]",same
28957,Van T. H. T.,Great experience. I loved it so much.,0.0,"10,0",NaN,Đánh giá cách đây 238 tuần,NaN,en,"[POS, POS]",same
28958,Ngan,We were able to be picked up at our own resort...,0.0,"9,0",NaN,Đánh giá cách đây 286 tuần,NaN,en,"[POS, POS]",same


In [31]:
df_labeled_en.to_csv ('english_reviews.csv')

In [41]:
# en_majority = df_labeled_en[df_labeled_en['labels'].apply(lambda x: x == ['NEG', 'NEU'])]
en_majority = df_labeled_en [df_labeled_en ['label_type'] == 'majority']
# en_majority.to_csv ('test.csv')
en_majority

,Author,Content,Likes,Per_Rating,Source,Time_Review,Title,language,labels,label_type
10,NaN,Food are decent... service was good...must giv...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 1 2024 trên KKday,Worth the price,en,"[POS, NEU]",majority
11,NaN,船上每個樓層，表演的節目各不相同，吃完buffet可以到樓層看表演，可惜船只有開到Bitex...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 8 2023 trên KKday,坐船遊西貢河，有趣的體驗,en,"[NEU, NEG]",majority
12,NaN,So so nothing special,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 7 2023 trên KKday,fair,en,"[NEG, NEU]",majority
21,NaN,The buffet dinner served was good but drinks n...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 4 2023 trên KKday,Great cruise,en,"[POS, NEU]",majority
30,NaN,"It was a good try, it just took awhile for the...",NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 3 2023 trên KKday,a nice experience,en,"[POS, NEU]",majority
...,...,...,...,...,...,...,...,...,...,...
28932,Trinh T. H.,"The experience was not that bad, but not excel...",0.0,"8,0",NaN,Đánh giá cách đây 289 tuần,NaN,en,"[POS, NEU]",majority
28934,Lee S. C.,It doesn't have a lot of animals and any anima...,0.0,"6,0",NaN,Đánh giá cách đây 290 tuần,NaN,en,"[NEU, NEG]",majority
28939,TOH S. Y.,We’re enjoyed feeding giraffes and the breakfa...,0.0,"9,0",NaN,Đánh giá cách đây 303 tuần,NaN,en,"[POS, NEU]",majority
28944,Dinh N. H.,It was ok. I don't really like that wild anima...,0.0,"8,0",NaN,Đánh giá cách đây 321 tuần,NaN,en,"[NEG, NEU]",majority


In [53]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
import torch

#### Load model
model_path = '5CD-AI/Vietnamese-Sentiment-visobert'
tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

sentence = 'Dịch vụ ok nhất là ăn uống, không quá mắc, hợp lý. Nhưng với số lượng các trò chơi như hiện tại thì mức giá 490k là quá mắc, không đáng lắm. 1đ trừ nữa là thu tiền giữ xe :)) với giá vé như vậy mà không free giữ xe thì em có hơi không vui, mặc dù không bao nhiêu nhưng cải thiện được sẽ tốt hơn nhiều lắm'
print('Sentence: ', sentence)

input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    out = model(input_ids)
    scores = out.logits.softmax(dim=-1).cpu().numpy()[0]

# Print labels and scores
ranking = np.argsort(scores)
ranking = ranking[::-1]
print("### Sentiment score ####")
for i in range(scores.shape[0]):
    l = config.id2label[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l}: {np.round(float(s), 4)}")

Sentence:  Dịch vụ ok nhất là ăn uống, không quá mắc, hợp lý. Nhưng với số lượng các trò chơi như hiện tại thì mức giá 490k là quá mắc, không đáng lắm. 1đ trừ nữa là thu tiền giữ xe :)) với giá vé như vậy mà không free giữ xe thì em có hơi không vui, mặc dù không bao nhiêu nhưng cải thiện được sẽ tốt hơn nhiều lắm
### Sentiment score ####
1) NEG: 0.9901
2) POS: 0.0056
3) NEU: 0.0043


In [79]:
# Load model and tokenizer once
model_path = '5CD-AI/Vietnamese-Sentiment-visobert'
tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

def predict_sentiment(sentence):
    # Tokenize input
    input_ids = torch.tensor([tokenizer.encode(sentence, truncation=True, max_length=512)])

    # Dự đoán
    with torch.no_grad():
        outputs = model(input_ids)
        scores = outputs.logits.softmax(dim=-1).cpu().numpy()[0]
    print (scores)
    # Lấy nhãn có xác suất cao nhất
    predicted_idx = np.argmax(scores)
    predicted_label = config.id2label[predicted_idx]
    return predicted_label

result = predict_sentiment ('The review was excellent. We have a lot of fun. Service excellent, food was good, Những khung cảnh đặc biệt, sóng nước hiền hòa của miền quê Việt Nam')
result

[0.00606357 0.02143706 0.9724993 ]


'NEU'

In [80]:
import torch
from transformers import RobertaForSequenceClassification, AutoTokenizer

model = RobertaForSequenceClassification.from_pretrained("wonrax/phobert-base-vietnamese-sentiment")

tokenizer = AutoTokenizer.from_pretrained("wonrax/phobert-base-vietnamese-sentiment", use_fast=False)

# Just like PhoBERT: INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
sentence = 'The review was excellent. We have a lot of fun. Service excellent, food was good, Những khung cảnh đặc biệt, sóng nước hiền hòa của miền quê Việt Nam .'  

input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    out = model(input_ids)
    print(out.logits.softmax(dim=-1).tolist())
    # Output:
    # [[0.002, 0.988, 0.01]]
    #     ^      ^      ^
    #    NEG    POS    NEU


[[0.0027723582461476326, 0.9901183247566223, 0.007109430618584156]]


In [81]:
# Load model và tokenizer 1 lần
checkpoint = "mr4/phobert-base-vi-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model.eval()

def predict_sentiment(sentence):
    # Tokenize đầu vào
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    
    # Dự đoán với mô hình
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]

    # Lấy nhãn có xác suất cao nhất
    predicted_label = model.config.id2label[probs.argmax().item()]
    
    return predicted_label

result = predict_sentiment ('The review was excellent. We have a lot of fun. Service excellent, food was good, Những khung cảnh đặc biệt, sóng nước hiền hòa của miền quê Việt Nam' )
result

'Tích cực'

In [117]:
df.to_csv ('test.csv')

In [116]:
from collections import Counter

# Hàm phân loại loại nhãn
def classify_label_type(label_list):
    counts = Counter(label_list)
    unique = len(counts)
    if unique == 1:
        return "same"
    elif unique == 2:
        return "majority"
    else:
        return "all_diff"

# Tạo cột phụ để phân loại
df['label_type'] = df['labels'].apply(classify_label_type)

# Tách thành 3 DataFrame
df_same = df[df['label_type'] == 'same'].copy()
df_majority = df[df['label_type'] == 'majority'].copy()
df_all_diff = df[df['label_type'] == 'all_diff'].copy()

In [118]:
df_same

,review,labels,label_type
1,Đội ngũ nhân viên luôn sẵn sàng giúp đỡ.,"[POS, POS, POS]",same
2,tệ ..đi 1 lần k có lần sau ..người ta muốn đi ...,"[NEG, NEG, NEG]",same
6,"Sạch và nhiều trò chơi trẻ con rất thích, lần ...","[POS, POS, POS]",same
9,OK mọi thứ luôn nhiệt tình và rất sạch thoáng ...,"[POS, POS, POS]",same
12,"Tuyệt vời, nhân viên hỗ trợ quá nhiệt tình, nế...","[POS, POS, POS]",same
...,...,...,...
15480,"Tour tạm ổn, đồ ăn trưa ngon, có bạn chụp hình...","[POS, POS, POS]",same
15482,Cứ phải gọi là tuyệt cà là vời. Hdv chụp hình ...,"[POS, POS, POS]",same
15483,Hướng dẫn viên D-1 có WA để đón giờ tại khách ...,"[POS, POS, POS]",same
15484,Nhân viên rất thân thiện và cũng nói được tiến...,"[POS, POS, POS]",same


In [119]:
df_majority

,review,labels,label_type
0,"Giá đặt online rẻ, nhiều trò chơi cảm giác mạn...","[POS, POS, NEU]",majority
4,"Khu nước:\nNhân viên nhiệt tình, chu đáo, dễ t...","[NEU, POS, NEU]",majority
5,Điểm nổi bật là nhà vệ sinh 100% đều đc nhân v...,"[POS, POS, NEU]",majority
8,"Mình có đặt vé tham quan ở Amazing Bay, mình c...","[NEU, POS, NEU]",majority
10,Nhân viên quầy vé thấy mua vé onl thì có mấy c...,"[NEG, NEU, NEG]",majority
...,...,...,...
15489,Trời mưa bão nhưng bên hãng nói vẫn đi được. S...,"[NEG, NEG, NEU]",majority
15491,Tàu dừng cách chỗ có san hô quá xa. San hô khô...,"[NEG, NEG, NEU]",majority
15493,Chuyến đi mình mong chờ nhất. Tuy nhiên thời g...,"[NEU, POS, NEU]",majority
15495,Trời mưa bão nhưng bên hãng nói vẫn đi được. S...,"[NEG, NEG, NEU]",majority


In [120]:
df_all_diff

,review,labels,label_type
3,"Trò chơi ít, không như tóp tóp review, tương đ...","[NEG, POS, NEU]",all_diff
7,"Dịch vụ ok nhất là ăn uống, không quá mắc, hợp...","[NEG, POS, NEU]",all_diff
59,mọi thứ vẫn tuyệt nhưng mà hôm nay mình đi mới...,"[NEU, POS, NEG]",all_diff
74,Mình đi team 7 người và cảm thấy số lượng trò ...,"[NEU, POS, NEG]",all_diff
75,"chơi vui, hơi rộng, ít trò nhưng mà chơi ok. c...","[NEG, POS, NEU]",all_diff
...,...,...,...
15461,Bạn có thể mua vé tại các cổng với giá tương đ...,"[NEG, POS, NEU]",all_diff
15486,Tour 2 đảo mà thấy lên được có 1 đảo. Có chươn...,"[NEG, POS, NEU]",all_diff
15488,"Ra đảo đúng ngày biển động, sóng to, gió lớn. ...","[NEG, POS, NEU]",all_diff
15492,Tour 2 đảo mà thấy lên được có 1 đảo. Có chươn...,"[NEG, POS, NEU]",all_diff


In [121]:
df_same.to_csv ('df_same.csv')
df_majority.to_csv ('df_majority.csv')
df_all_diff.to_csv ('df_all_diff.csv')

In [ ]:
"Đây là một câu review du lịch. Dựa trên nội dung, nếu phải chọn giữa 3 nhãn sau: POS, NEG, NEU, bạn chọn cái nào? Gợi ý: 2 model chọn POS, 1 model chọn NEG."

In [ ]:
import pandas as pd
from IPython.display import display
import ipywidgets as widgets

# Giả sử bạn có df_all_diff là DataFrame chưa rõ nhãn
index = 0
final_labels = []

def on_button_click(btn):
    global index
    final_labels.append(btn.description)
    index += 1
    if index < len(df_all_diff):
        text.value = df_all_diff.iloc[index]["review"]
    else:
        print("Xong rồi!")
        text.value = "Đã hết dữ liệu để gán nhãn."

text = widgets.Textarea(
    value=df_all_diff.iloc[index]["review"],
    disabled=True,
    layout=widgets.Layout(width='90%', height='100px')
)

buttons = [widgets.Button(description=l, button_style="") for l in ['POS', 'NEG', 'NEU']]
for btn in buttons:
    btn.on_click(on_button_click)

display(text, widgets.HBox(buttons))


In [ ]:
import gradio as gr

def label_sentiment(review, label):
    return f"Label '{label}' được chọn cho review:\n{review}"

gr.Interface(
    fn=label_sentiment,
    inputs=[
        gr.Textbox(lines=5, label="Review"),
        gr.Radio(['POS', 'NEG', 'NEU'], label="Chọn nhãn")
    ],
    outputs="text"
).launch()


# Gọi LLMs xử lý các reviews mâu thuẫn

In [63]:
from together import Together
import pandas as pd
from tqdm import tqdm

# 🔑 Set API Key
client = Together(api_key="34eaf9f075946ac73b7df627b7ca9592dd4d0c63244eb75038b2876761179bde")

# 📌 Prompt phân tích cảm xúc tiếng Việt
def build_sentiment_prompt_vi(review_text):
    return f"""
You are a sentiment analysis assistant. Analyze the sentiment of the following review written in Vietnamese about a tourism experience. Classify it into one of the following sentiment labels:

- POS (positive)
- NEG (negative)
- NEU (neutral or mixed)

Only return one of the three labels: POS, NEG, or NEU. Do not translate the review. Do not explain. Only return the label.

Review (in Vietnamese):
\"\"\"
{review_text}
\"\"\"

Sentiment label:
""".strip()

# 🧠 Hàm gọi Together LLaMA
def get_sentiment_label_llama(review_text):
    try:
        prompt = build_sentiment_prompt_vi(review_text)
        response = client.chat.completions.create(
            model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",  # Có thể thay bằng 8b nếu muốn nhanh hơn
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=5,
        )

        output = response.choices[0].message.content.strip().upper()
        if output not in ['POS', 'NEG', 'NEU']:
            return 'NEU'
        return output
    except Exception as e:
        print("[Lỗi LLaMA]", review_text[:50], e)
        return 'NEU'

def label_majority_reviews_with_llama(df, text_column="Content"):
    df = df.copy()

    # Chỉ lọc các dòng có label_type == 'majority'
    mask = df['label_type'] == 'majority'
    df_majority = df[mask].copy()

    # Áp dụng gán nhãn llama_label bằng LLaMA
    tqdm.pandas(desc="Labeling with LLaMA")
    df_majority["llama_label"] = df_majority[text_column].progress_apply(get_sentiment_label_llama)

    # Gộp lại vào DataFrame gốc
    df.loc[df_majority.index, "llama_label"] = df_majority["llama_label"]

    return df

df_labeled = label_majority_reviews_with_llama(df_labeled)

Labeling with LLaMA: 100%|██████████| 2858/2858 [4:09:11<00:00,  5.23s/it]  


In [64]:
from together import Together
import pandas as pd
from tqdm import tqdm

# 🔑 Set API Key
client = Together(api_key="34eaf9f075946ac73b7df627b7ca9592dd4d0c63244eb75038b2876761179bde")

# 📌 Prompt phân tích cảm xúc tiếng Việt
def build_sentiment_prompt_vi(review_text):
    return f"""
You are a sentiment analysis assistant. Analyze the sentiment of the following review written in Vietnamese about a tourism experience. Classify it into one of the following sentiment labels:

- POS (positive)
- NEG (negative)
- NEU (neutral or mixed)

Only return one of the three labels: POS, NEG, or NEU. Do not translate the review. Do not explain. Only return the label.

Review (in English):
\"\"\"
{review_text}
\"\"\"

Sentiment label:
""".strip()

# 🧠 Hàm gọi Together LLaMA
def get_sentiment_label_llama(review_text):
    try:
        prompt = build_sentiment_prompt_vi(review_text)
        response = client.chat.completions.create(
            model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",  # Có thể thay bằng 8b nếu muốn nhanh hơn
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=5,
        )

        output = response.choices[0].message.content.strip().upper()
        if output not in ['POS', 'NEG', 'NEU']:
            return 'NEU'
        return output
    except Exception as e:
        print("[Lỗi LLaMA]", review_text[:50], e)
        return 'NEU'

def label_majority_reviews_with_llama(df, text_column="Content"):
    df = df.copy()

    # Chỉ lọc các dòng có label_type == 'majority'
    mask = df['label_type'] == 'majority'
    df_majority = df[mask].copy()

    # Áp dụng gán nhãn llama_label bằng LLaMA
    tqdm.pandas(desc="Labeling with LLaMA")
    df_majority["llama_label"] = df_majority[text_column].progress_apply(get_sentiment_label_llama)

    # Gộp lại vào DataFrame gốc
    df.loc[df_majority.index, "llama_label"] = df_majority["llama_label"]

    return df

df_labeled_english = label_majority_reviews_with_llama(df_labeled_en)

Labeling with LLaMA: 100%|██████████| 681/681 [57:16<00:00,  5.05s/it]  


In [72]:
a = df_labeled [df_labeled['label_type'] == 'majority']
a

,Author,Content,Likes,Per_Rating,Source,Time_Review,Title,language,labels,label_type,llama_label
9,NaN,Đồ ăn ngon. Dắt tour nhiệt tình. Nhiều thứ để ...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 10 2019 trên KKday,khách hàng,vi,"[POS, NEU, POS]",majority,POS
13,NaN,Đây là khu vui chơi mới nên tất cả còn tô...,NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 7 2024 trên KKday,khu vui chơi giải trí,vi,"[POS, POS, NEG]",majority,NEU
14,NaN,Rất nhiều trò chơi thú vị. nhưng phần tham qua...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 7 2024 trên KKday,Rất nhiều trò chơi thú vị,vi,"[NEG, NEU, NEU]",majority,NEU
23,NaN,Mặc dù Tết là khá đông nhưng nhân viên vẫn sup...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 2 2024 trên KKday,Đánh giá,vi,"[POS, POS, NEU]",majority,POS
24,NaN,"Chuyến đi là trải nghiệm mới cho mình và các bạn,",NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 12 2023 trên KKday,Tương đối,vi,"[POS, POS, NEU]",majority,POS
...,...,...,...,...,...,...,...,...,...,...,...
10067,Tram A. T.,Tour rất ổn. Tuy có hơi gấp một chút nhưng mìn...,1.0,"10,0",NaN,Đánh giá cách đây 239 tuần,NaN,vi,"[NEU, POS, POS]",majority,NEU
10069,Truong T. T. T.,Tàu dừng cách chỗ có san hô quá xa. San hô khô...,4.0,"6,0",NaN,Đánh giá cách đây 289 tuần,NaN,vi,"[NEG, NEG, NEU]",majority,NEU
10071,Leo R.,Chuyến đi mình mong chờ nhất. Tuy nhiên thời g...,6.0,"10,0",NaN,Đánh giá cách đây 294 tuần,NaN,vi,"[NEU, POS, NEU]",majority,NEU
10073,Thanh T. N.,Trời mưa bão nhưng bên hãng nói vẫn đi được. S...,0.0,"4,0",NaN,Đánh giá cách đây 300 tuần,NaN,vi,"[NEG, NEG, NEU]",majority,NEG


In [74]:
from together import Together
import pandas as pd
from tqdm import tqdm

# 🔑 Set API Key
client = Together(api_key="34eaf9f075946ac73b7df627b7ca9592dd4d0c63244eb75038b2876761179bde")

# 📌 Prompt phân tích cảm xúc tiếng Việt
def build_sentiment_prompt_vi(review_text):
    return f"""
You are a sentiment analysis assistant. Analyze the sentiment of the following review written in Vietnamese about a tourism experience. Classify it into one of the following sentiment labels:

- POS (positive)
- NEG (negative)
- NEU (neutral or mixed)

Only return one of the three labels: POS, NEG, or NEU. Do not translate the review. Do not explain. Only return the label.

Review (in Vietnamese):
\"\"\"
{review_text}
\"\"\"

Sentiment label:
""".strip()

# 🧠 Hàm gọi Together LLaMA
def get_sentiment_label_llama(review_text):
    try:
        prompt = build_sentiment_prompt_vi(review_text)
        response = client.chat.completions.create(
            model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",  # Có thể thay bằng 8b nếu muốn nhanh hơn
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=5,
        )

        output = response.choices[0].message.content.strip().upper()
        if output not in ['POS', 'NEG', 'NEU']:
            return 'NEU'
        return output
    except Exception as e:
        print("[Lỗi LLaMA]", review_text[:50], e)
        return 'NEU'

def label_majority_reviews_with_llama(df, text_column="Content"):
    df = df.copy()

    # Chỉ lọc các dòng có label_type == 'majority'
    mask = df['label_type'] == 'all_diff'
    df_majority = df[mask].copy()

    # Áp dụng gán nhãn llama_label bằng LLaMA
    tqdm.pandas(desc="Labeling with LLaMA")
    df_majority["llama_label"] = df_majority[text_column].progress_apply(get_sentiment_label_llama)

    # Gộp lại vào DataFrame gốc
    df.loc[df_majority.index, "llama_label"] = df_majority["llama_label"]

    return df

df_labeled_full = label_majority_reviews_with_llama(df_labeled)

Labeling with LLaMA: 100%|██████████| 339/339 [29:12<00:00,  5.17s/it]


In [75]:
df_labeled_full

,Author,Content,Likes,Per_Rating,Source,Time_Review,Title,language,labels,label_type,llama_label
0,NaN,Mọi thứ đều rất bài bản và tuyệt. Xứng đáng vớ...,NaN,5/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 5 2023 trên KKday,Dịch vụ tuyệt vời,vi,"[POS, POS, POS]",same,NaN
1,NaN,"Thức ăn ngon, dịch vụ tại thuyền tốt. Thuyền h...",NaN,4/5 trên KKday,Do KKday thu thập,Nhận xét vào Tháng 7 2019 trên KKday,tham quan,vi,"[POS, POS, POS]",same,NaN
2,NaN,Tour được tổ chức chu đáo. Anh hướng dẫn viên ...,NaN,5/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 6 2024 trên Tripadvisor,Great tour!,vi,"[POS, POS, POS]",same,NaN
3,NaN,Có một chuyến đi rất vui và nhiều niềm vui cùn...,NaN,5/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 2 2023 trên Tripadvisor,Chuyến đi gia đình cùng hướng dẫn viên Bảo,vi,"[POS, POS, POS]",same,NaN
4,NaN,Anh Lộc (lockie) hướng dẫn rất vui tính và tận...,NaN,5/5 trên Tripadvisor,Do Tripadvisor thu thập,Nhận xét vào Tháng 8 2022 trên Tripadvisor,Tour tuyệt vời,vi,"[POS, POS, POS]",same,NaN
...,...,...,...,...,...,...,...,...,...,...,...
10071,Leo R.,Chuyến đi mình mong chờ nhất. Tuy nhiên thời g...,6.0,"10,0",NaN,Đánh giá cách đây 294 tuần,NaN,vi,"[NEU, POS, NEU]",majority,NEU
10072,Ba Quan M.,"Ra đảo đúng ngày biển động, sóng to, gió lớn. ...",4.0,"5,0",NaN,Đánh giá cách đây 297 tuần,NaN,vi,"[NEG, POS, NEU]",all_diff,NEU
10073,Thanh T. N.,Trời mưa bão nhưng bên hãng nói vẫn đi được. S...,0.0,"4,0",NaN,Đánh giá cách đây 300 tuần,NaN,vi,"[NEG, NEG, NEU]",majority,NEG
10074,Freshi M.,"Tài xế rất nhiệt tình, trước khi đón H-1 tài x...",0.0,"10,0",NaN,Đánh giá cách đây 34 tuần,NaN,vi,"[POS, POS, POS]",same,NaN


In [76]:
df_labeled_full.to_csv ('vietnamese_reviews_dataset.csv')

In [79]:
df_labeled_english.to_csv ('english_reviews_dataset.csv')